# Homework 1 
## ME 326: Collaborative Robotics
### Stanford University

## Problem 1: Non-Holonomic Robot Control


![](https://armlabstanford.github.io/figures/me326/homework1/nonholonomicrobot_2.jpeg)

### Kinematics

Here we have a non-holonomic (meaning its linear velocity is restricted so that the forward velocity points along $\hat{r}_{1}$). The position of reference frame $R$ is $^A{r}^R=x_R \hat{
a}_1 + y_R \hat{a}_2$, its velocity $^A{v}^R = \dot{x}_R \hat{a}_1 + \dot{y}_R \hat{a}_2$ and must respect the non-integrable, knife-edge non-holonomic constraint meaning $^R{v}=v\hat{r}_1 + 0 \hat{r}_2 + 0\hat{r}_3$ and for a constraint vector $^A\lambda = -sin(\theta)\hat{a}_1 + cos(\theta)\hat{a}_2$, the nonholonomic constraint is written as: 

$$-sin(\theta)\dot{x}_R + cos(\theta) \dot{y}_R = 0$$

For the rotation from frame $R$ to frame $A$, the rotation is about the axis $\hat{a}_1$ through angle $\theta$:

$$ \begin{bmatrix}\hat{a}_1 \\ \hat{a}_2 \\ \hat{a}_3   \end{bmatrix} = \underbrace{\begin{bmatrix} cos(\theta) & -sin(\theta) & 0 \\ sin(\theta) & cos(\theta) & 0 \\ 0 & 0 & 1 \end{bmatrix}}_{^AR_R} \begin{bmatrix} \hat{r}_1 \\ \hat{r}_2 \\ \hat{r}_3 \end{bmatrix} $$

Due to the holonomic constraint, if a desired point $^A{r}^D(t)$ were to lie on the non-holonomic constraint vector $^A\lambda$ then the immediate velocity of the robot could not move in that direction due to the constraint. One strategy would be to rotate in-place first until the $\hat{r}_{1}$ axis were aligned with the target position. While this strategy would work well if the goal were static, if we take the case where the goal is on a circle with the robot at the center as was moving on the circle such that it stayed on the non-holonomic line as the robot rotated then the robot would simply rotate in place at the center of the circle. 

But if we consider this scenario, we intuitively know that the robot could apply a small forward velocity and essentially meet the desired point on the circle by following the path of the spiral until it reached the circle containing the desired point. 

*To achieve this, we must consider the control of a point that does not sit on the non-holonomic constraint line (as $R$ frame sits on the non-holonomic constraint line)*. Let's consider a point P (shown in the figure, fixed in frame $R$) that sits at $^Rr^P=L \hat{r}_1 + 0 \hat{r}_2 + 0 \hat{r}_3$. This point in frame A is: 

$$^Ar^P = ^Ar^R + [^AR_R] ^Rr^P$$
Differentiating we have: 
$$^A{\dot{r}}^P = ^A{\dot{r}}^R + [^A{\dot{R}}_R] ^Rr^P + [^AR_R] ^R{\dot{r}}^P$$
we know by definition that $^R{\dot{r}}^P=0$, and 

$$[^A{\dot{R}}_R] = [^A \omega ^R]^{[\times]}[^AR_R]$$ 

where $^A\omega ^R = 0\hat{a}_1 + 0 \hat{a}_2 + \dot{\theta} \hat{a}_3$. We also have that  $^A{\dot{r}}^R = [^A R_R] ^R{v}$:
$$^A{\dot{r}}^R = ^A R_R \begin{bmatrix} v\hat{r}_1 \\ 0 \hat{r}_2 \\ 0\hat{r}_3  \end{bmatrix}  $$

Substituting these above we get a solution of the form
$$^A{\dot{r}}^P = [^A R_R] ^R{v} + [^A \omega ^R]^{[\times]}[^AR_R] ^Rr^P $$
If we simplify this to only include the xy-position in the A frame (remove the z-component) we can simplify to: 
$$\begin{bmatrix} 
^A{\dot{r}}^P_x \\ ^A{\dot{r}}^P_y
\end{bmatrix} = M_{2\times 2} \begin{bmatrix} 
v \\ \dot{\theta}
\end{bmatrix} $$

This is the matrix you will show is full rank (non-zero determinant), and use for control of the xy-position in the next part: 

#### Functions
1. You will make the function that returns the xy-position of point P: $^A{{r}}^P_{xy}$ in a $2\times1$ numpy matrix (e.g. ```np.matrix([[2][4]]) = np.matrix([2,4]).T```) given $^A{{r}}^R_{xy}$, $L$ and $\theta$:
$$^A{{r}}^P_{xy} = f_1(^A{{r}}^R_{xy}, L,\theta) $$
2. You will make a function that calculate the above matrix $M$ ($2\times 2$) given $L$, $\theta$:
$$ M_{2\times 2} = f_2(L,\theta)$$

2D numpy matrix: ```np.matrix([[1,2],[3,4]]) ``` $=\begin{bmatrix}1 & 2 \\ 3 & 4 \end{bmatrix}$ (if you need to get the inverse at some point later: $M^{-1}$: ```M_inv = np.linalg.inv(M)```)

### Control

For control, you will be given a 2D xy-position point to track $^Ar^D_{xy}(t) = x_d(t) \hat{a}_1 + y_d(t) \hat{a}_2 $, the input to this function will also be the current state of the robot $^A{{r}}^R_{xy}(t)$ and angle $\theta$. In this function you will want to 
- Obtain the point P: $^A{{r}}^P_{xy}(t)$ leveraging your earlier function and $^A{{r}}^R_{xy}(t)$, $L$, $\theta$
- Find the xy-error: $^A e(t) = ^Ar^D_{xy}(t) - ^A{{r}}^P_{xy}(t)$
- Obtain the current matrix $M$: $ M_{2\times 2}(t) = f_2(L,\theta(T))$
- Calculate your control law, I suggest something along the lines of 
$$u_{2\times 1}(t)= \begin{bmatrix}v \\ \dot{\theta} \end{bmatrix}_{2\times 1}= M^{(-1)}_{2\times 2}[K_p]_{2\times 2} [^A e_{xy}]_{2\times 1}(t)$$ 
where $[K_p]_{2\times 2} = \begin{bmatrix} K & 0 \\ 0 & K \end{bmatrix}$ for $K>0$.

#### Function
1. You will write a controller that provides a control law: 
$$ u_{2\times 1}(t) = f_3([^Ar^D]_{xy}(t), [^A{{r}}^R]_{xy}(t), \theta(t), L)$$
with the output being a $2\times 1$ numpy matrix.

## Alternate Expression for Kinematics
You can show the the velocity of the robot frame can be represented in the following way:
$$\begin{bmatrix}^A\dot{x}_R \\ ^A\dot{y}_R \\ \dot{\theta}  \end{bmatrix} = \begin{bmatrix} cos(\theta) & 0 \\
sin(\theta) & 0 \\
0 & 1\end{bmatrix} \underbrace{\begin{bmatrix} v \\ \dot{\theta} \end{bmatrix}}_{u(t)} $$


## Top-Level

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation
from matplotlib import rc
rc('animation', html='jshtml')
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import solve_ivp
%matplotlib notebook

In [ ]:
# parameters
max_time = 60
dt = 1. / 20.
t = np.linspace(0, max_time, int(max_time / dt) + 1)
L = 0.01

## Your Code

In [ ]:
def student_function_1(r, theta, L):
    """Returns the xy position of point P in frame A

    Parameters
    ----------
    r : np.ndarray, shape=(2,)
        Current position of frame R with respect to frame A.
    theta : float
        Current rotation angle about axis a_3.
    L : float
        Position of P in frame R along axis r_1.

    Returns
    -------
    p : np.ndarray, shape=(2,)
        Position of P in frame A.
    """
    rP_R = np.array([L, 0])

    # Rotation matrix from frame R to frame A
    R_AR = np.array(
        [[np.cos(theta), -np.sin(theta)],
        [np.sin(theta),  np.cos(theta)]]
    )

    # Transform point P from frame R to frame A
    p = r + R_AR @ rP_R
    return p


def student_function_2(L, theta):
    """Returns the matrix M.

    Parameters
    ----------
    See student_function_1.

    Returns
    -------
    M : np.ndarray, shape=(2, 2)
        The matrix M relating input velocities to the velocity of point P.
    """
    M = np.array(
        [[np.cos(theta), -L * np.sin(theta)],
         [np.sin(theta),  L * np.cos(theta)]]
    )
    return M


def student_function_3(rd, r, L, theta):
    """Computes the control input u=[v, omega]^T.

    Parameters
    ----------
    rd : np.ndarray, shape=(2,)
        Desired position of robot position.
    See student_function_1 for rest of parameters.

    Returns
    -------
    u : np.ndarray, shape=(2,)
        Control input [v, omega]^T.
    """
    p = student_function_1(r, theta, L)

    e = rd - p

    M = student_function_2(L, theta)

    K = 1.0
    K_p = np.array(
        [[K, 0],
        [0, K]]
    )
    M_inv = np.linalg.inv(M)

    u = M_inv @ K_p @ e
    
    return u

## Simulation

In [ ]:
class Simulation(object):
    def __init__(self, rd_type='circle'):
        assert rd_type == 'circle' or rd_type == 'infinity'
        self.rd_type = rd_type

    def desired_position_circle(self,time):
        ang_rate = 0.15 * np.pi / 2.0
        circ_rad = 2.0
        rd = np.array([circ_rad * np.cos(ang_rate * time), circ_rad * np.sin(ang_rate * time)])
        return rd

    def desired_position_infinity(self, time):
        ang_rate = 0.4 * np.pi / 3.0
        circ_rad = 2
        x_lim = 10 
        x_rate = 0.5
        #rd = np.array([time, circ_rad * np.sin(ang_rate * time) + 0.3])
        rd = np.array([circ_rad * np.sin(0.5*ang_rate * time), circ_rad * np.sin(ang_rate * time) + 0.1])

        return rd

    def nonholonomic_kinematics(self, tsim, state, L=0.1):
        x1, x2, x3 = state
        r = np.array([x1,x2])
        if self.rd_type == 'circle':
            rd = self.desired_position_circle(tsim)
        elif self.rd_type == 'infinity':
            rd = self.desired_position_infinity(tsim)
        else:
            raise NotImplementedError

        theta = x3
        u = student_function_3(rd, r, L, theta)
        v = u.item(0)
        dth = u.item(1)
        dx1 = np.cos(x3) * v
        dx2 = np.sin(x3) * v
        dx3 = dth
        return [dx1, dx2, dx3]

    def integrator(self, times, L, x0=None):
        
        if type(x0) == type(None):
            x0 = np.array([0, 0, 0]) #set this if it hasn't been set
        dyn = lambda t, x: self.nonholonomic_kinematics(t, x, L=L)
        sol = solve_ivp(dyn, [times[0], times[-1]], x0, t_eval=times)
        x_t = sol.y.T

        if self.rd_type == 'circle':
            x_d_t = np.array(
                [
                    [self.desired_position_circle(time).item(0),self.desired_position_circle(time).item(1)]
                    for time in t
                ]
            )
        elif self.rd_type == 'infinity':        
            x_d_t = np.array(
                [
                    [self.desired_position_infinity(time).item(0),self.desired_position_infinity(time).item(1)]
                    for time in t
                ]
            )
        else:
            raise NotImplementedError
        return [x_t, x_d_t]

def plot_result(x_t, x_d_t):
    fig = plt.figure()
    ax = fig.add_subplot(111, autoscale_on=True)
    ax.plot(x_t[:, 0], x_t[:, 1], label=r'$^Ar^R$', c='b')
    ax.plot(x_d_t[:, 0], x_d_t[:, 1], label=r'$^Ar^D$', c='r')
    ax.grid()
    ax.legend()

## Plot the result


In [ ]:
#Follow a infinity curve
cls_obj = Simulation('infinity')
x_t, x_d_t = cls_obj.integrator(t, L)
plot_result(x_t, x_d_t)

In [ ]:
#Follow a circle curve
cls_obj = Simulation('circle')
x_t, x_d_t = cls_obj.integrator(t, L)
plot_result(x_t, x_d_t)

## Animate Results:

In [ ]:
from matplotlib import animation
from IPython.display import HTML

#Draw the objects
fig3 = plt.figure();
ax3 = fig3.add_subplot(111, autoscale_on=False, xlim=(-10, 10), ylim=(-10, 10));
ax3.set_aspect('equal')

#define animation structions
robot_frame, = ax3.plot([], [], '-',color='green', lw=3)
robot_left_tire, = ax3.plot([], [], '-',color='blue', lw=1)
robot_right_tire, = ax3.plot([], [], '-',color='blue', lw=1)
robot_des_frame, = ax3.plot([], [], 'o',color='red', lw=15)

def update_robot(i, robot_frame, robot_left_tire, robot_right_tire):
    robot_x = x_t[i, 0]
    robot_y = x_t[i, 1]
    theta = x_t[i, 2]

    rx = [-0.2, 0.2]
    ry = [-0.2, 0.2]
    wxl = [-0.215, -0.210]
    wxr = [0.21, 0.215]
    wy = [-0.2, 0.2]

    wl = np.array([[robot_x + np.cos(theta) * wxx, robot_y + np.sin(theta) * wyy] for wxx in wxl for wyy in wy])
    wr = np.array([[robot_x + np.cos(theta) * wxx,robot_y + np.sin(theta) * wyy] for wxx in wxr for wyy in wy])

    r = np.array([[robot_x + np.cos(theta) * rxx, robot_y + np.sin(theta) * ryy] for rxx in rx for ryy in ry])

    # robot frame
    robot_frame_x_ = [-0.2, 0.2, 0.2, -0.2, -0.2]
    robot_frame_y_ = [-0.2, -0.2, 0.2, 0.2, -0.2]
    robot_frame_x = [
        robot_x + np.cos(theta) * robot_frame_x_[j] - np.sin(theta) * robot_frame_y_[j] for j in range(5)
    ]
    robot_frame_y = [
        robot_y + np.sin(theta) * robot_frame_x_[j] + np.cos(theta) * robot_frame_y_[j] for j in range(5)
    ]

    # robot left tire
    robot_left_tire_x_ = [-0.215, 0.210]
    robot_left_tire_y_ = [-0.2, -0.2]
    robot_left_tire_x = [
        robot_x + np.cos(theta) * robot_left_tire_x_[j] - np.sin(theta) * robot_left_tire_y_[j] for j in range(2)
    ]
    robot_left_tire_y = [
        robot_y + np.sin(theta) * robot_left_tire_x_[j] + np.cos(theta) * robot_left_tire_y_[j] for j in range(2)
    ]

    # robot right tire
    robot_right_tire_x_ = [-0.2, 0.2] 
    robot_right_tire_y_ = [0.21, 0.21]
    robot_right_tire_x = [
        robot_x + np.cos(theta) * robot_right_tire_x_[j] - np.sin(theta) * robot_right_tire_y_[j] for j in range(2)
    ]
    robot_right_tire_y = [
        robot_y + np.sin(theta) * robot_right_tire_x_[j] + np.cos(theta) * robot_right_tire_y_[j] for j in range(2)
    ]

    # return robot frame
    robot_frame.set_data(robot_frame_x,robot_frame_y)
    robot_left_tire.set_data(robot_left_tire_x, robot_left_tire_y)
    robot_right_tire.set_data(robot_right_tire_x, robot_right_tire_y)
    ax3.set_title(f"Time: {(i * dt):0.2f}")
    
    # desired point
    p_des_x = x_d_t[i, 0]
    p_des_y = x_d_t[i, 1]
    robot_des_frame.set_data([p_des_x], [p_des_y])
    
    return robot_frame, robot_left_tire, robot_right_tire, robot_des_frame

robot_ani = animation.FuncAnimation(
    fig3,
    update_robot,
    len(t),
    fargs=(robot_frame, robot_left_tire, robot_right_tire),
    interval=10,
)



Now show the animation, note this can take about a minute to run

In [ ]:
robot_ani

## Problem 2: State Estimation (Kalman Filtering) for Non-holonomic Robot 

### State Estimation Under Uncertainty - Kalman Filtering
If we have a process model (kinematic/dynamic) of our system (Problem 1), what do we do if our process model is imperfect or our measurement model is noisy or inconsistent? Our goal would be to approximate the expected state, but from a safety perspective it is very important to also model our uncertainty (or our confidence in our state estimate). 

This can be achieved through the state filtering which iterates between finding the next expected state given the current state and control inputs and then leveraging available measurements to improve the estimated state. A very common filtering method to achieve this is the ***Kalman Filter***.

***Kalman Filter Algorithm***

Given a state and time $t$: $s_t$, multivariate uncertainty matrix $\Sigma_t$. For a system dynamics and observation models described by: $$s_{t} = A_t s_{t-1} + B_t u_{t}$$ and $$z_{t}=C_t s_{t}$$ then the Kalman Filtering algorithm has the following steps: 

*Process Update*

1. Update State Dynamics: $$\bar{s}_{t} = A_t s_{t-1} + B_t u_{t}$$
2. Update Uncertainty (process noise $R_t$): $$\bar{\Sigma}_t = A_t \Sigma_{t-1} A_t^T + R_t$$

*Measurement Update*

3. Kalman Gain (measurement noise: $Q_t$): $$K_t = \bar{\Sigma}_t C_t^T(C_t\bar{\Sigma}_tC_t^T + Q_t)^{-1}$$
4. State Estimate: $$s_t = \bar{s}_{t} + K_t(z_t - C_t \bar{s}_{t})$$
5. Uncertainty Estimate: $$\Sigma_t = (I - K_t C_t)\bar{\Sigma}_t$$


### Applying Filtering to the Motion of the Non-Holonomic Base
For the base, we have already derived for a control point P: 
$$\begin{bmatrix} 
^A{\dot{r}}^P_x \\ ^A{\dot{r}}^P_y
\end{bmatrix} = M_{2\times 2} \begin{bmatrix} 
v \\ \dot{\theta}
\end{bmatrix} $$

If we define for simplicity $s = ^A{r}^P$ and $\dot{s} = ^A{\dot{r}}^P$, then the dynamics can be found by simple integration: $$s_{t+\Delta t} = \underbrace{A}_{I} s_{t} + \Delta t \underbrace{\dot{s}_{t}}_{M_{2\times 2} \begin{bmatrix} 
v \\ \dot{\theta}
\end{bmatrix} }.$$

The observation model will be defined to let us see the entire state: $$z_{t} = \underbrace{C}_{I} s_t.$$

*In the following functions, the student will implement both the process and measurement updates of the Kalman Filter. A Governing script will call both functions, intermittently disrupting measurements in which case state uncertainty is expected to grow until it is reduced by measurement observations $z_t$.*

#### Functions
1. You will make the state process update function that returns the estimated state and variance $(\bar{s}_{t},\bar{\Sigma}_t)$ for a given control input (relating to state derivative), initial state and initial variance $(\underbrace{\dot{s}_{t-1}}_{M_{2\times 2} \begin{bmatrix} 
v \\ \dot{\theta}
\end{bmatrix} }, s_{t-1},\bar{\Sigma}_{t-1})$. The instructor governing script will provide input for the previous student function that will generate $u_{2\times 1}(t) = f_3([^Ar^D]_{xy}(t), [^A{{r}}^R]_{xy}(t), \theta(t), L)$. 


2. You will make a measurement update function that will compute a) the Kalman Gain $K_t$ b) the state estimate $s_t$ and c) the uncertainty estimate $\Sigma_t$. 


## Your Code

In [ ]:
def student_function_4(cov, process_noise, dt, rd, r, L, theta):
    """Returns the process update state and covariance of the xy position of point P in frame A
    
    Parameters
    ----------
    cov : np.ndarray, shape=(2, 2)
        The covariance matrix Sigma for uncertainty in point P (x,y) position.
        
    process_noise : np.ndarray, shape=(2, 2)
        The process noise matrix R_t for uncertainty in the process.
    
    dt : float, scalar
        Time step for process model integration (defined in Problem 1)
    
    See student_function_3 for rest of parameters. Current state is r.
    
    Returns
    -------
    r_bar : np.ndarray, shape=(2,)
        Process update estimate of position P in frame A.
    cov_bar: np.ndarray, shape=(2, 2)
        Process update estimate of covariance matrix Sigma for uncertainty in point P (x,y) position.
    """
    #First obtain u, M to obtain ds/dt; then integrate dynamics for the process model:
    #Find the covariance    
    u = student_function_3(rd, r, L, theta)

    M = student_function_2(L, theta)

    # need the state derivative ds/dt = M  * u 
    ds_dt = M @ u

    r_bar = r + ds_dt * dt # double check this

    A = np.eye(2)
    cov_bar = A @ cov @ A.T + process_noise


    return [r_bar,cov_bar]
    
def student_function_5(z_obs, r_bar, cov_bar, meas_noise):
    """Returns the measurement update state and covariance of the xy position of point P in frame A
    
    Parameters
    ----------
    z_obs : np.ndarray, shape=(2,)
        Measurement of position of point P in frame A.
    r_bar : np.ndarray, shape=(2,)
        Process update estimate of position P in frame A.
    cov_bar : np.ndarray, shape=(2, 2)
        Process update estimate of covariance matrix Sigma for uncertainty in point P (x,y) position.
    meas_noise : np.ndarray, shape=(2, 2)
        The measurement noise matrix Q_t for uncertainty in the measurement.
    
    Returns
    -------
    r_est : np.ndarray, shape=(2,)
        Measurement update estimate of position P in frame A.
    cov_est: np.ndarray, shape=(2, 2)
        Measurement update estimate of covariance matrix Sigma for uncertainty in point P (x,y) position.
    """
    #First find the Kalman Gain
    #Update the state estimate
    #Update the uncertainty estimate
    C = np.eye(2)

    if meas_noise.ndim == 1:
        meas_noise = np.diag(meas_noise)
    
    # gain calculation
    innov_cov = C @ cov_bar @ C.T + meas_noise
    K = cov_bar @ C.T @ np.linalg.inv(innov_cov)

    # state update
    innov = z_obs - C @ r_bar
    r_est = r_bar + K @ innov

    # covariance update
    I = np.eye(2)
    cov_est = (I - K @ C) @ cov_bar


    return [r_est,cov_est]

## Simulation

In [ ]:
class KF_simulation(object):
    def __init__(self, rd_type='circle'):
        assert rd_type == 'circle' or rd_type == 'infinity'
        self.rd_type = rd_type
        self.sim_cls_obj = Simulation(rd_type) 
        
    def find_des_r(self,tsim):
        if self.rd_type == 'circle':
            rd = self.sim_cls_obj.desired_position_circle(tsim)
        elif self.rd_type == 'infinity':
            rd = self.sim_cls_obj.desired_position_infinity(tsim)
        else:
            raise NotImplementedError
        return rd
        
    def KF_integrator(self,times, dt=0.05, L=0.01, meas_prob = 0.6):
        '''
        meas_prob: probability that a measurement is taken, number btw 0:(never measured), 1:(always measured)         
        '''
        #This function intializes and rolls out the true and estimated state
        #Initialize state and covariance (uncertainty) and set process noise
        x0 = np.array([0, 0, 0])
        s = x0[:2]
        init_variance = 3 #initial uncertainty variance
        cov = init_variance*np.identity(2)
        
        #delta_x_per_dt = 0.01*dt #this is the uncertainty in the process model for 1 second of rollout divided by the specified time step  
        delta_x_per_dt = 0.2*dt #this is the uncertainty in the process model for 1 second of rollout divided by the specified time step  
        
        
        Rt = delta_x_per_dt*np.identity(2)
        #Find the true state with the given dynamics (over the whole trajectory)
        x_t_true, x_d_t_true = self.sim_cls_obj.integrator(t, L,x0=x0)
        # Make the (noisy measurements - simulating a real system)
        
        #meas_noise_std = 0.05 #picked this rad of traj is ~2
        meas_noise_std = 0.05 #picked this rad of traj is ~2
        
        z_obs = np.array([m[:2] + np.random.normal(0,meas_noise_std,2) for m in x_t_true]) #measurement with added noise
        Qt = meas_noise_std*np.ones(2)
        
        #make a list of values
        x_t_est = [s]
        cov_est = [cov]
        for idx in range(len(t)-1):
            #Iterate through time, with process updates and measurement updates (when allowed based on meas_probability)
            #Set terms:
            theta = x_t_true[idx][2]#For simplicity, we will provide ground truth theta from the oracle
            r = s #here we set the state to the estimated state
            rd = self.find_des_r(t[idx])
            process_noise = Rt
            #call the process update
            s,cov = student_function_4(cov, process_noise, dt, rd, r, L, theta)
            #determine if a measurement can be obtained:
            meas_bool = np.random.choice([True,False],replace=True,p=[meas_prob,1-meas_prob]) #used to determine if measurement is made in this loop
            if meas_bool:
                #call Measurement update
                z_obs_curr = z_obs[idx]
                r_bar = s
                cov_bar = cov
                meas_noise = Qt
                s,cov = student_function_5(z_obs_curr, r_bar, cov_bar, meas_noise)

            x_t_est.append(s)
            cov_est.append(cov)
        #convert to ndarray
        x_t_est = np.array(x_t_est)
        cov_est = np.array(cov_est)
        return [x_t_est, cov_est, x_t_true, x_d_t_true]
        
def plot_KF_result(x_t_est, cov_est, x_t_true, x_d_t_true,times,traj_type='Circle'):
    #take the diagonal elements of the variance matrix for time vs x (or y) plots
    covx = np.array([c[0][0] for c in cov_est])
    covy = np.array([c[1][1] for c in cov_est])
    #plot the x-axis data
    figx = plt.figure()
    ax1 = figx.add_subplot(111, autoscale_on=True)
    ax1.plot(times,x_t_true[:, 0], label=r'$^Ar_x^R$', c='b')
    ax1.plot(times,x_d_t_true[:, 0], label=r'$^Ar_x^D$', c='r')
    ax1.plot(times,x_t_est[:, 0], label=r'$^Ar_x^E$', c='g') #estimated
    
    ax1.fill_between(times, x_t_est[:, 0] - covx, x_t_est[:, 0] + covx, color='gray', alpha=0.2)
    
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('$x$ position')
    ax1.set_title('Trajectory: '+traj_type)
    ax1.grid()
    ax1.legend()   
    
    #plot the y-axis data
    figy = plt.figure()
    ax2 = figy.add_subplot(111, autoscale_on=True)
    ax2.plot(times, x_t_true[:, 1], label=r'$^Ar_y^R$', c='b')
    ax2.plot(times, x_d_t_true[:, 1], label=r'$^Ar_y^D$', c='r')
    ax2.plot(times, x_t_est[:, 1], label=r'$^Ar_y^E$', c='g') #estimated
    
    ax2.fill_between(times, x_t_est[:, 1] - covy, x_t_est[:, 1] + covy, color='gray', alpha=0.2)
    
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('$y$ position')
    ax2.set_title('Trajectory: '+traj_type)
    ax2.grid()
    ax2.legend()
    

## Plot the result

In [ ]:
#Follow a infinity curve with Kalman filtering
cls_obj_kf = KF_simulation('infinity')
x_t_est, cov_est, x_t_true, x_d_t_true = cls_obj_kf.KF_integrator(t, dt=dt, L=L,meas_prob = 0.02) 
plot_KF_result(x_t_est, cov_est, x_t_true, x_d_t_true,t,traj_type='infinity')

In [ ]:
#Follow a infinity curve with Kalman filtering
cls_obj_kf = KF_simulation('circle')
x_t_est, cov_est, x_t_true, x_d_t_true = cls_obj_kf.KF_integrator(t, dt=dt, L=L,meas_prob = 0.02) #0.05
plot_KF_result(x_t_est, cov_est, x_t_true, x_d_t_true,t,traj_type='circle')
#now draw ellipse: https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.patches.Ellipse.html

## Animate Results: 
pause previous animation before running

In [ ]:
from matplotlib import animation
from IPython.display import HTML


#Draw the objects
fig4 = plt.figure();
ax4 = fig4.add_subplot(111, autoscale_on=False, xlim=(-10, 10), ylim=(-10, 10));
ax4.set_aspect('equal')

#define animation structions
robot_frame, = ax4.plot([], [], '-',color='green', lw=3)
robot_left_tire, = ax4.plot([], [], '-',color='blue', lw=1)
robot_right_tire, = ax4.plot([], [], '-',color='blue', lw=1)
robot_des_frame, = ax4.plot([], [], 'o',color='red', lw=15)
robot_est_frame, = ax4.plot([], [], '*',color='yellow', lw=15)

kf_robot_uncertainty, = ax4.plot([], [], '.',color='grey', lw=1)



def update_robot(i, robot_frame, robot_left_tire, robot_right_tire):
    robot_x = x_t_true[i, 0]
    robot_y = x_t_true[i, 1]
    theta = x_t_true[i, 2]

    rx = [-0.2, 0.2]
    ry = [-0.2, 0.2]
    wxl = [-0.215, -0.210]
    wxr = [0.21, 0.215]
    wy = [-0.2, 0.2]

    wl = np.array([[robot_x + np.cos(theta) * wxx, robot_y + np.sin(theta) * wyy] for wxx in wxl for wyy in wy])
    wr = np.array([[robot_x + np.cos(theta) * wxx,robot_y + np.sin(theta) * wyy] for wxx in wxr for wyy in wy])

    r = np.array([[robot_x + np.cos(theta) * rxx, robot_y + np.sin(theta) * ryy] for rxx in rx for ryy in ry])

    # robot frame
    robot_frame_x_ = [-0.2, 0.2, 0.2, -0.2, -0.2]
    robot_frame_y_ = [-0.2, -0.2, 0.2, 0.2, -0.2]
    robot_frame_x = [
        robot_x + np.cos(theta) * robot_frame_x_[j] - np.sin(theta) * robot_frame_y_[j] for j in range(5)
    ]
    robot_frame_y = [
        robot_y + np.sin(theta) * robot_frame_x_[j] + np.cos(theta) * robot_frame_y_[j] for j in range(5)
    ]

    # robot left tire
    robot_left_tire_x_ = [-0.215, 0.210]
    robot_left_tire_y_ = [-0.2, -0.2]
    robot_left_tire_x = [
        robot_x + np.cos(theta) * robot_left_tire_x_[j] - np.sin(theta) * robot_left_tire_y_[j] for j in range(2)
    ]
    robot_left_tire_y = [
        robot_y + np.sin(theta) * robot_left_tire_x_[j] + np.cos(theta) * robot_left_tire_y_[j] for j in range(2)
    ]

    # robot right tire
    robot_right_tire_x_ = [-0.2, 0.2] 
    robot_right_tire_y_ = [0.21, 0.21]
    robot_right_tire_x = [
        robot_x + np.cos(theta) * robot_right_tire_x_[j] - np.sin(theta) * robot_right_tire_y_[j] for j in range(2)
    ]
    robot_right_tire_y = [
        robot_y + np.sin(theta) * robot_right_tire_x_[j] + np.cos(theta) * robot_right_tire_y_[j] for j in range(2)
    ]

    # return robot frame
    robot_frame.set_data(robot_frame_x,robot_frame_y)
    robot_left_tire.set_data(robot_left_tire_x, robot_left_tire_y)
    robot_right_tire.set_data(robot_right_tire_x, robot_right_tire_y)
    ax4.set_title(f"Time: {(i * dt):0.2f}")
    
    # desired point
    p_des_x = x_d_t_true[i, 0]
    p_des_y = x_d_t_true[i, 1]
    robot_des_frame.set_data([p_des_x], [p_des_y])
    
    # draw estimate state and covariance ellipse
    robot_est_frame.set_data([x_t_est[i,0]],[x_t_est[i,1]])

    cc = cov_est[i]
    [U,S,V] = np.linalg.svd(cc)
    cov_height = S[0]
    cov_width = S[1]
    th = np.arctan2(U[1,0],U[0,0])
    
    u=x_t_est[i,0]     #x-position of the center
    v=x_t_est[i,1]    #y-position of the center
    a=cov_height     #radius on the x-axis
    b=cov_width    #radius on the y-axis
    epts = np.linspace(0, 2*np.pi, 100)
    ep_x = u + np.cos(th)*a*np.cos(epts) - np.sin(th)*b*np.sin(epts)
    ep_y = v + np.sin(th)*a*np.cos(epts) + np.cos(th)*b*np.sin(epts)
    
    kf_robot_uncertainty.set_data([ep_x],[ep_y])
    
    
    return robot_frame, robot_left_tire, robot_right_tire, robot_des_frame, robot_est_frame, kf_robot_uncertainty

robot_ani2 = animation.FuncAnimation(
    fig4,
    update_robot,
    len(t),
    fargs=(robot_frame, robot_left_tire, robot_right_tire),
    interval=10,
)

Now display the animation:

In [ ]:
robot_ani2

## Problem 3: Forward Kinematics of 2-Link Manipulator

![FK_example](https://armlabstanford.github.io/figures/me326/homework1/FK_example.png)

Given the 2-Link manipulator as shown in the figure, where frame $A$ is the fixed world frame, frame $B$ is allowed to rotate with angular velocity: 
$$^{A}{\omega}^{B} = 0 \hat{a}_{1} + 0 \hat{a}_{2} + \dot{\theta}_1\hat{a}_{3}$$
Frame $C$ as seen by $B$ is described by the vector fixed in $B$:
$$^{B}r^C = 0 \hat{b}_{1} + L_1 \hat{b}_{2} + 0 \hat{b}_{3}$$
Frame $C$ rotates with respect to frame $B$ with 
$$ ^{B}\omega^{C} = \dot{\theta}_2 \hat{b}_{1} + 0 \hat{b}_{2} + 0 \hat{b}_{3}$$
And frame $D$ position as seen by $C$ is described by the vector fixed in C:
$$^{C}{{r}}^{D} = 0 \hat{c}_{1} + 0 \hat{c}_{2} + L_2 \hat{c}_{3} $$
Point ${P}$ is the point that sits at the origin of frame $D$

1. Express the forward kinematics function (FK):
$$^{A}{P} = FK(\theta_1, \theta_2, L_1, L_2)$$

2. Evaluate and plot the above function ($FK(\cdot)$ for every combination of $\theta_1 \in [0,2\pi]$, $\theta_2 \in [0,2\pi]$
at a resolution of at least $\Delta \theta_i = \frac{10^{\circ} \pi}{180^{\circ}}$. 

*Hint:* 
1. *Make a function that evaluates the forward kinematics for the end-effector point P, then* 
2. *Make a function that cycles through $\theta_{1,i}, \theta_{2,j}$ and makes a list of all the points $P_{ij}$.* 
3. *Use matplotlib 3D plot to plot scatter points (if you reason about the orientation of the revolute joints, you will expect to see a Torus shape mapped by the end-effector!)*

## Top Level

In [1]:
import itertools  # hint: itertools.product might be useful
import numpy as np
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
from matplotlib import animation, rc
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import solve_ivp

%matplotlib notebook

In [2]:
# values of theta1 and theta2 at which you should plot the location of point P
theta1_list = np.linspace(0, 2 * np.pi, 37)
theta2_list = np.linspace(0, 2 * np.pi, 37)

## Your Code

In [3]:
def forward_kinematics(theta1, theta2, L1, L2):
    # Rotation Matrix from A to B 
    R_A_B  = np.array(
        [[np.cos(theta1), -np.sin(theta1), 0],
        [np.sin(theta1),  np.cos(theta1), 0],
        [0,               0,              1]]
    )

    # position of C in frame B
    r_B_C = np.array([0, L1, 0])

    # rotation Matrix from B to C
    R_B_C = np.array(
        [[1,  0,               0              ],
        [0,  np.cos(theta2), -np.sin(theta2)],
        [0,  np.sin(theta2),  np.cos(theta2)]]
    )

    # position of D in frame C 
    r_C_D = np.array([0, 0, L2])

    # position of B in frame A
    r_A_B = np.array([0, 0, 0])

    # C --> B -> A 
    r_A_C = r_A_B + R_A_B @ r_B_C

    # R --> A -- C
    R_A_C = R_A_B @ R_B_C

    # position of D in frame A
    r_A_D = r_A_C + R_A_C @ r_C_D

    P = r_A_D
    return P 


In [4]:
def evaluate_kinematics(theta1_list, theta2_list, L1, L2):
    points = []

    for theta1 in theta1_list:
        for theta2 in theta2_list:
            P = forward_kinematics(theta1, theta2, L1, L2)
            points.append(P)
    return np.array(points)


## Plot the result

In [11]:
import plotly.express as px
import pandas as pd

# lenghts of the links
L1 = 1.0
L2 = 1.0

points = evaluate_kinematics(theta1_list, theta2_list, L1, L2)
df = pd.DataFrame(
    {
        'x': points[:, 0],
        'y': points[:, 1],
        'z': points[:, 2],
    }
)

fig = px.scatter_3d(
    df,
    x='x',
    y='y',
    z='z',
    title='Workspace of 2-Link Manipulator',
    labels={'x': 'X', 'y': 'Y', 'z': 'Z'},
    opacity=0.5,
    size_max=3
)


fig.update_layout(
    scene=dict(
        #aspectmode='data',
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        bgcolor='white'
    ),
    width=1080,
    height=720
)

fig.show()